In [1]:
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
%ls

LJ_data/                        high-dimension-paper-results/
LJ_data.tar                     high-dimension-search-results/
README.md                       images/
__pycache__/                    jobs/
algorithm_compare_results/      low-dimension-search-results/
config.py                       models/
data/                           paper-images/
data-queue/                     requirements.txt
data-queue-2023-09-24/          results/
data-queue-2023-09-27/          search_results/
deeplifting/                    tasks.py
deeplifting.png                 test-low-dimension-results.png


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.models import DeepliftingSkipMLP
from deeplifting.optimization import deeplifting_predictions
from deeplifting.utils import initialize_vector

In [9]:
problem = PROBLEMS_BY_NAME['bukin_n6']
output_size = problem['dimensions']

# Set up the deeplifting model
input_size = 1
hidden_sizes = (128,) * 5
bounds = problem['bounds']
activation = 'sine'
output_activation = 'leaky_relu'
agg_function = 'sum'
trial = 0

model = DeepliftingSkipMLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
    bounds=None,
    skip_every_n=1,
    activation=activation,
    output_activation=output_activation,
    agg_function=agg_function,
    seed=trial,
)

# Need to setup an objective
results = np.zeros((1, 1, 3))
trial = 0

objective = problem['objective']
fn = lambda x: objective(x, results=results, trial=trial, version='pytorch')

In [10]:
inputs = torch.randn(1, 5 * output_size).to(dtype=torch.double)
model = model.to(dtype=torch.double)
outputs = model(inputs=inputs)
outputs.flatten()

tensor([2.4555, 2.3634], dtype=torch.float64, grad_fn=<ReshapeAliasBackward0>)

In [11]:
x0 = initialize_vector(size=output_size, bounds=bounds)
x0 = torch.from_numpy(x0)
x0 = x0.to(dtype=torch.double)
print(x0)

model.output_layer.linear.weight.data = torch.zeros_like(
    model.output_layer.linear.weight.data
)
model.output_layer.batch_norm.weight.data = torch.zeros_like(
    model.output_layer.batch_norm.weight.data
)
model.output_layer.linear.bias = torch.nn.Parameter(x0)

tensor([-9.5119,  1.2911], dtype=torch.float64)


In [12]:
model(inputs=inputs).mean(axis=0)

tensor([-0.0951,  1.2911], dtype=torch.float64, grad_fn=<MeanBackward1>)

In [13]:
for name, parameters in model.named_parameters():
    if 'output' in name:
        print(name, parameters)

output_layer.linear.weight Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 